**Neuro-AI: Harnessing AI to understand computation in mind and brain**  
**Day 3: Neural encoding, decoding & reconstruction with deep learning**  

# Reconstructing handwritten digits from fMRI recordings

In tutorial, you will implement a neural decoder for reconstructing perceived stimuli from brain responses.

The dataset contains fMRI data acquired from the early visual cortex of one subject as the subject was presented with 100 grayscale images of handwritten sixes and nines (50 sixes and 50 nines). The fMRI data has been realigned and slice time corrected. Furthermore, stimulus specific response amplitudes have been estimated with a general linear model.

Let's first familiarize ourselves with the dataset. It contains a number of variables:

* **X** (stimuli) -> This is a 100 x 784 matrix. Each ith row contains the pixel values of a stimulus that was presented in the ith trial of the experiment. Note that the stimuli are 28 pixel x 28 pixel images, which were reshaped to 1 x 784 vectors.

* **Y** (brain data) -> This is a 100 x 3092 matrix. The ith row contains the voxel values of the responses that were measured in the ith trial of the experiment.

* **X_prior** (auxiliary images) -> This is a 2000 x 784 matrix. Each row contains the pixel values of a different stimulus, which was _not_ used in the experiment. Note that the stimuli are 28 pixel x 28 pixel images, which were reshaped to 1 x 784 vectors. You will be needing **X_prior** in task 3.

Note: In the remainder of this document, we will use **x** for referring to a 784 x 1 stimulus vector and **y** for referring to a 3092 x 1 response vector.

## Task 1

* Load the dataset.
* Visualize some of the stimuli.
* Normalize X and Y and X_prior to have zero mean and unit variance. Tip: Recall that normalization means subtracting the mean of each pixel/voxel from itself and dividing it by its standard deviation. Also save these means and standard deviations as you will need them to denormalize the predictions again, for visualization purposes.
* Split X and Y in two parts called X_training and X_test, and Y_training and Y_test. The training set should contain 80 stimulus-response pairs (40 pairs for sixes and 40 pairs for nines -- use the labels to evenly split these two categories). The test set should contain 20 stimulus-response pairs (10 pairs for sixes and 10 pairs for nines). Note that label 1 and 2 correspond to sixes and nines, respectively.

In [ ]:
!wget -nc https://github.com/tdado/NeuroAI24/raw/main/Basic/69dataset.mat

import random
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import stats
from scipy.stats import t

# Load the dataset
data = scipy.io.loadmat("69dataset.mat")
print(data.keys())

X = data["X"]              # Shape: (100, 784)
Y = data["Y"]              # Shape: (100, 3092)
X_prior = data["prior"]    # Shape: (2000, 784)
labels = data["labels"]    # Shape: (100, 1)

File ‘69dataset.mat’ already there; not retrieving.

dict_keys(['__header__', '__version__', '__globals__', 'structural', 'funloc', 'masks', 'prior', 'labels', 'Y', 'X', 'description'])


In [ ]:
### Your code here

## Task 2

Our goal is to solve the problem of reconstructing **x** from **y**. One possible approach to solve this problem is to use a *discriminative* model. Discriminative models predict **x** as a function of **y**. That is:

$y = f(x)$

We will assume that f is a linear function. That is:

$x = B^\top y$

$f$ can be seen as a very simple linear neural network comprising one layer of weights (i.e., $B$). The aim of ridge regression is to minimize the objective function:

$L(B) = \|X - YB\|^2 + \lambda \|B\|^2$

$B$ can be estimated in closed form:

$B = (Y_{tr}^\top Y_{tr} + \lambda I)^{-1} Y_{tr}^\top X_{tr}$

where $\lambda$ is the regularization coefficient, $I$ is the $q \times q$ identity matrix, and $q$ is the number of voxels. Note that we can safely ignore the intercept since we normalized our data to have zero mean and unit variance.

* Provide a derivation of the above closed-form solution for $B$ by setting the derivative of $L(B)$ to zero and find the optimal $B$.
* Estimate $B$ on the training set. Tip: Normally, you should use cross validation to estimate lambda. For simplicity, you can assume that $\lambda = 10^{-6}$.
* Reconstruct $x$ from $y$ in the test set.
* Visualize the reconstructions.

ToThink 1: This model predicted the pixel values of an image by taking a linear combination of the stimulus-evoked brain responses. Does this imply that the relationship between responses and pixels of handwritten digits is inherently linear?

In [ ]:
### Your code here

## Task 3.1

Another possible approach to solve the problem of reconstructing **x** from **y** is to use a *generative* model and invert it with Bayes' rule. We reformulate the problem as finding the most probable **x** that could have caused **y**. That is:

argmax_**x** P(**x** | **y**)

where P(**x** | **y**) is called the posterior (probability of the stimulus being **x** if the observation is **y**). In other words, we have to define the posterior, estimate its parameters and find the argument that maximizes it, which will be the reconstruction of **x** from **y**. While, this may seem daunting, it actually has a simple solution. The posterior assigns a probability to an event by combining our observations and beliefs about it, and can be decomposed with Bayes' rule as the product of how likely our observations are given the event (probability of observing **y** if the stimulus is **x**) and how likely the event is independent of our observations (probability of the stimulus being **x**). That is:

P(**x** | **y**) ~ P(**y** | **x**) * P(**x**)

where P(**y** | **x**) is called the likelihood and P(x) is called the prior.

We will assume that the likelihood and the prior are multivariate Gaussian distributions. A Gaussian is characterized by two parameters: a mean vector and a covariance matrix. In the case of the likelihood, the mean of the Gaussian is given by:

**mu**\_likelihood = **B'** **x**

As before, we can estimate **B** in close form with ridge regression:

**B** = inv(**X**\_training' **X**\_training + lambda **I**) **X**\_training' **Y**\_training

where lambda is the regularization coefficient, I is the *p* x *p* identity matrix, and *p* is the number of pixels. The covariance matrix of the likelihood is given by:

**Sigma**_likelihood = diag(E[||**y** - **B'** **x**|| ^ 2]).

In the case of the prior, the mean of the Gaussian is given by:

**mu**\_prior = **0** (which is a vector of zeros)

The covariance matrix of the prior is given by:

**Sigma**\_prior = **X**\_prior' * **X**\_prior / (n - 1)

where n is the length of **X**\_prior.

* Estimate **B** on the training set. Tip: Normally, you should use cross-validation to estimate lambda and Sigma_likelihood. For simplicity, you can assume that lambda = 10 ^ -6 and Sigma_likelihood = 10 ^ -3 **I**.
* Estimate **Sigma**\_prior. Tip: Add 10 ^ -6 to the diagonal of Sigma_prior for regularization.
* Visualize **Sigma**\_prior.

ToThink 2: What does the visualized **Sigma**\_prior shows? Can you also visualize the covariance of one pixel (take one in the middle) -- what does this tell you?

In [ ]:
### Your code here

## Task 3.2

Having defined the likelihood and the prior as Gaussians, we can derive the posterior by multiplying them. It turns out that the product of two Gaussians is another Gaussian, whose mean vector is given by:

**mu**\_posterior = inv(inv(**Sigma**\_prior) + **B** inv(**Sigma**\_likelihood) **B**') **B** * inv(**Sigma**\_likelihood) **y**

We are almost done. Recall that the reconstruction of **x** from **y** is the argument that maximizes the posterior, which we derived to be a Gaussian. We will be completely done once we answer the following question: What is the argument that maximizes a Gaussian?

.  
.  
.  
.  
.  
.

The answer is its mean vector, which is the solution of our initial problem. That is:

argmax_**x** P(**x** | **y**) =  
**mu**\_posterior =  
inv(inv(**Sigma**\_prior) + **B** inv(**Sigma**\_likelihood) **B**') **B** * inv(**Sigma**\_likelihood) **y**

ToThink 3: Why does the mean vector of the posterior distribution represent the most probable $x$ given $y$?

Now, we can plug any **y** in the above equation and reconstruct the most probable **x** that could have caused it.

- Reconstruct **x** from **y** in the test set.
- Visualize the reconstructions.

ToThink 4: Compare the reconstructions with the earlier reconstructions. Which one is better? Why? Can you think of ways to improve the results?

In [ ]:
### Your code here

## Task 5

Quantify the reconstruction performance in terms of pixel similarity: the Pearson correlation coefficient between predicted and actual pixel values in the stimuli and reconstructions, respectively.

ToThink 5: what is the downside of this metric? Can you propose a more human-centric metric to assess reconstruction performance?

In [ ]:
def pearson_correlation_coefficient(x: np.ndarray, y: np.ndarray, axis: int) -> np.ndarray:
    r = (np.nan_to_num(stats.zscore(x)) * np.nan_to_num(stats.zscore(y))).mean(axis)
    p = 2 * t.sf(np.abs(r / np.sqrt((1 - r ** 2) / (x.shape[0] - 2))), x.shape[0] - 2)
    return r, p

In [ ]:
### Your code here